In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import gzip
!pip install json_lines
import json_lines
!pip install ipython-autotime
%load_ext autotime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=8d86fa322f535e03c926de39777f8b2b7ba201ed7ed68a2885446814dc64e3ef
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


# Loading Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 21.1 s


Both the original json files and the file I have collated are json line files. This means that rather than having an array of projects, each project is an indiviual json, separated by a new line (\n). I've used the json_lines library to parse each line and add it into an array.

In [5]:
#reading json_lines zipped file
file = '/content/drive/My Drive/Data Science Projects/Kickstarter/downloader/data url/collated.json.gz'

with gzip.open('/content/drive/My Drive/Data Science Projects/Kickstarter/downloader/data url/collated.json.gz','rb') as zip_file:
  json_data = [json.loads(line) for line in zip_file.readlines()]  

time: 38.5 s


In [7]:
type(json_data[0])

dict

time: 2.25 ms


In [8]:
#flattening and normalizing for dataframe.
df = pd.json_normalize(json_data)

time: 2min 16s


In [10]:
del(json_data)

time: 2.26 s


In [9]:
df.shape
#(315775, 100)

(315775, 100)

time: 2.28 ms


In [0]:
#quick_look_at_columns and contents
for title,value1,value2 in zip(df.columns,df.iloc[0],df.iloc[1]):
  print(f'COLUMN NAME: {title}\n{value1}\n{value2}\n\n')

COLUMN NAME: id
1670966448
294655417


COLUMN NAME: name
LoKi MUT - Your EDC multi-tool with 14 tools
WeirdWoodWear


COLUMN NAME: blurb
The LoKi MUT is a small EDC multitool that includes the key features required in a everyday carry tool.
Handmade wooden ear plugs and stretchers in any size and any type of wood, combined with fluorescent colors.


COLUMN NAME: goal
2000.0
500.0


COLUMN NAME: pledged
2474.0
0.0


COLUMN NAME: state
successful
failed


COLUMN NAME: slug
loki-mut
weirdwoodwear


COLUMN NAME: disable_communication
False
False


COLUMN NAME: country
US
AT


COLUMN NAME: country_displayable_name
the United States
Austria


COLUMN NAME: currency
USD
EUR


COLUMN NAME: currency_symbol
$
€


COLUMN NAME: currency_trailing_code
True
False


COLUMN NAME: deadline
1486096663
1450430554


COLUMN NAME: state_changed_at
1486096663
1450430555


COLUMN NAME: created_at
1480697214
1447687147


COLUMN NAME: launched_at
1483504663
1447838554


COLUMN NAME: staff_pick
False
False


COLU

In [0]:
columns_to_keep = ['id''name','blurb','goal','pledged','state','disable_communication','country','currency','deadline',
'state_changed_at','created_at','launched_at','staff_pick','static_usd_rate','converted_pledged_amount',
'converted_pledged_amount','location.name','location.type',
'category.name','category.parent_name','category.slug','profile.show_feature_image','profile.should_show_feature_image_section',
'spotlight','urls.web.project','creator.id','creator.name']
columns_to_remove = [x for x in df.columns if x not in columns_to_keep]

time: 3.02 ms


In [0]:
df.drop(columns_to_remove,axis=1,inplace = True)

time: 5.94 s


In [0]:
df.head()

,blurb,goal,pledged,state,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,staff_pick,static_usd_rate,converted_pledged_amount,spotlight,creator.id,creator.name,location.name,location.type,category.name,category.slug,category.parent_name,profile.show_feature_image,profile.should_show_feature_image_section,urls.web.project
0,The LoKi MUT is a small EDC multitool that inc...,2000.0,2474.0,successful,False,US,USD,1486096663,1486096663,1480697214,1483504663,False,1,2474.0,True,872303310,Travis stevenson,Dallas,Town,Gadgets,technology/gadgets,Technology,False,True,https://www.kickstarter.com/projects/872303310...
1,Handmade wooden ear plugs and stretchers in an...,500.0,0.0,failed,False,AT,EUR,1450430554,1450430555,1447687147,1447838554,False,1.06609,0.0,False,1539029550,Lucas Neumann,Micheldorf,Town,Woodworking,crafts/woodworking,Crafts,False,True,https://www.kickstarter.com/projects/153902955...
2,"Broke, But Not Broken LP #1",3000.0,4085.0,successful,False,US,USD,1578934880,1578934880,1575923302,1576256480,False,1,4085.0,True,1882152174,Michael Kane,Worcester,Town,Music,music,NaN,False,True,https://www.kickstarter.com/projects/themornin...
3,Maker Box is an AR play experience that combin...,10000.0,10353.0,successful,False,US,USD,1480715804,1480715804,1476326585,1478120204,False,1,10353.0,True,1031612808,Manoj Jacob,Los Gatos,Town,Children's Books,publishing/children's books,Publishing,False,True,https://www.kickstarter.com/projects/getquriou...
4,A children's book about a robots search for lo...,11000.0,14862.0,successful,False,US,USD,1465185660,1465185660,1460864401,1461905683,True,1,14862.0,True,1480798481,Joshua Margolis,Oakland,Town,Children's Books,publishing/children's books,Publishing,False,True,https://www.kickstarter.com/projects/148079848...


time: 59.9 ms


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315775 entries, 0 to 315774
Data columns (total 25 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   blurb                                      315775 non-null  object 
 1   goal                                       315775 non-null  float64
 2   pledged                                    315775 non-null  float64
 3   state                                      315775 non-null  object 
 4   disable_communication                      315775 non-null  bool   
 5   country                                    315775 non-null  object 
 6   currency                                   315775 non-null  object 
 7   deadline                                   315775 non-null  int64  
 8   state_changed_at                           315775 non-null  int64  
 9   created_at                                 315775 non-null  int64  
 10  launched

The table above shows that:
- date fields such as deadline were casted as integers
- static_usd_rate should be float type

# Data Wrangling

**To do**
- Analyse missing values
- Analyse data types
- Turn dates into datetime format
- Estimate project duration in days

- Convert different currencies into USD
- Scrape:
  - Rewards
  - Text Length
  - Number of videos
  - Number of images
  - Number of links
  - Number of updates
- Translate Blurb
- Eliminate non-completed projects (state)
- Replace state column with 'completed' column
- drop remaining NAs

## Missing Values

In [0]:
#Item 1 Analyse Missing Values
df.isna().sum()

blurb                                             0
goal                                              0
pledged                                           0
state                                             0
disable_communication                             0
country                                           0
currency                                          0
deadline                                          0
state_changed_at                                  0
created_at                                        0
launched_at                                       0
staff_pick                                        0
static_usd_rate                                   0
converted_pledged_amount                      51692
spotlight                                         0
creator.id                                        0
creator.name                                      0
location.name                                  1051
location.type                                  1051
category.nam

time: 651 ms


#### Converted Pledged Amount
This value represents the amount pledged in USD. This field can be calculated by multiplying 'pledged' by 'static_usd_rate'. Since there are no missing values among these last two fields, I can recalculate the 'converted_pledged_amount' with all the information I need.

In [0]:
df[['pledged','static_usd_rate','converted_pledged_amount']].head(30)

,pledged,static_usd_rate,converted_pledged_amount
0,2474.00,1,2474.0
1,0.00,1.06609,0.0
2,4085.00,1,4085.0
3,10353.00,1,10353.0
4,14862.00,1,14862.0
5,8592.00,1,8592.0
6,8100.00,1,8100.0
7,27470.29,0.751315,21045.0
8,11.00,1.26323,14.0
9,270.00,1.4041,377.0


time: 26.5 ms


In [0]:
df.drop('converted_pledged_amount',axis=1,inplace = True)

time: 157 ms


#### Missing Locations

All the missing locations are within the US. I haven't yet decided if I will only be using the country, or I may try to classify the location_names into categories based on the population numbers. 

In [0]:
df[df['location.name'].isna()].country.value_counts()

US    1051
Name: country, dtype: int64

time: 71.7 ms


#### Missing Category Parent Names
Almost half of the values in this columns are missing. A quick look into the data shows that there there is more than a single reason why these values are missing.

In [0]:
parent_names = df['category.parent_name'].unique()
print(parent_names)

['Technology' 'Crafts' nan 'Publishing' 'Music' 'Food' 'Film & Video'
 'Fashion' 'Photography' 'Design' 'Theater' 'Dance' 'Journalism' 'Art'
 'Games' 'Comics']
time: 73.1 ms


In [0]:
# Category names with missing parent names
print(df[df['category.parent_name'].isna()]['category.name'].unique())

['Music' 'Journalism' 'Technology' 'Design' 'Fashion' 'Publishing' 'Food'
 'Theater' 'Comics' 'Art' 'Photography' 'Crafts' 'Dance' 'Film & Video'
 'Games' 'Apps' 'Tabletop Games' 'Accessories' 'Illustration' 'Software'
 'Playing Cards' 'Gadgets' 'Apparel' 'Pop' 'Web' 'Country & Folk'
 'Live Games' 'Webseries' 'Comic Books' 'Graphic Design' 'Animation'
 'Woodworking' 'Shorts' 'Public Art' 'Comedy' "Children's Books"
 'Painting' 'Video Games' 'Performances' 'Mobile Games' 'Mixed Media'
 'Art Books' 'Fiction' 'Festivals' 'Print' 'Product Design'
 'Space Exploration' 'Drama' 'Graphic Novels' 'Hip-Hop' 'Hardware'
 'Events' 'Classical Music' 'Narrative Film' 'Music Videos'
 'DIY Electronics' 'Food Trucks' 'Restaurants' 'Nonfiction' 'Documentary'
 'Anthologies' 'Sculpture' 'Drinks' 'Zines' 'Musical' 'Rock' 'Jazz'
 'Sound' 'Footwear' 'Robots' 'Indie Rock' 'Science Fiction' 'Jewelry'
 'Photobooks' 'Puzzles' 'Nature' 'Performance Art' 'Spaces' 'Academic'
 'Poetry' 'Periodicals' 'People' 'Calenda

**Parent name not indexed on json file**

I can clearly see that some the data wasn't correctly indexed during the scraping. However, the category.slug field contains the information that I need.

I will write a function that firstly checks if the project is missing the parent name and whether it can be found on on the 'category.slug' field. 

To ensure I'm not collecting the wrong data, I will add a condition by which only already existing names will be added to the parent name field (eg: Music, Theatre, etc).

In [0]:
df[(df['category.name'] == 'Jazz') &(df['category.parent_name'].isna())][['category.name','category.parent_name','category.slug']].head()

,category.name,category.parent_name,category.slug
163248,Jazz,NaN,music/jazz
164397,Jazz,NaN,music/jazz
171047,Jazz,NaN,music/jazz
172910,Jazz,NaN,music/jazz
173012,Jazz,NaN,music/jazz


time: 121 ms


In [0]:
def parent_fixer(project):
  if isinstance(project['category.parent_name'],str):
    return project['category.parent_name']
  else:
    try:
      prnt = project['category.slug'].split('/')[0].title()
      if prnt in parent_names:
        return prnt
      else: 
        return project['category.parent_name']
    except:
      return project['category.parent_name']



df['category.parent_name'] = df.apply(parent_fixer, axis = 1)

print(df['category.parent_name'].isna().sum())

0
time: 9.9 s


Great news! There are no missing values remaining!

In [0]:
 # Category names with missing parent names
for parent in df['category.parent_name'].unique():
  print(df[df['category.parent_name'] == parent].groupby(['category.name','category.slug'])['category.name'].count())

category.name      category.slug               
3D Printing        technology/3d printing           807
Apps               technology/apps                 7746
Camera Equipment   technology/camera equipment      554
DIY Electronics    technology/diy electronics      1176
Fabrication Tools  technology/fabrication tools     308
Flight             technology/flight                436
Gadgets            technology/gadgets              3977
Hardware           technology/hardware             4062
Makerspaces        technology/makerspaces           285
Robots             technology/robots                701
Software           technology/software             3414
Sound              technology/sound                 931
Space Exploration  technology/space exploration     375
Technology         technology                      1637
Wearables          technology/wearables            1580
Web                technology/web                  4270
Name: category.name, dtype: int64
category.name  categor

In [0]:
# for id in df[df['category.parent_name'].isna()].index:
#   df['category.parent_name'].loc[id] = df['category.name'].loc[id]
#   df['category.name'].loc[id] = 'Generic'

time: 1.31 ms


In [0]:
# !pip install googletrans
# #Item 1 - Language Detection
# from googletrans import Translator
# translator = Translator()

# def translate(translator,list_of_strings):
#   language = []
#   translation = []
#   for text in list_of_strings:
#     try:
#       trans = translator.translate(text)
#       language.append(trans.src)
#       if trans.src!= 'en':
#         translation.append(trans.text)
#       else:
#         translation.append(text)
#     except:
#       language.append(np.NaN)
#       translation.append(np.NaN)
#   return language, translation

# df['blurb.language'], df['blurb.translation'] = translate(translator,df.blurb)


## Dates

In [0]:
#turning dates fields into datetime format
for field in ['deadline','state_changed_at','created_at','launched_at']:
  df[field] = pd.to_datetime(df[field],unit='s')

time: 45.7 ms


## Project Duration

In [0]:
df['duration'] = df.apply(lambda x:(x['deadline']-x['launched_at']).days,axis =1)

time: 18.9 s


## Currencies

I'm going to use the static_usd_rate to convert all goals and pledged amouns into dollars.

One challenge I had was that for some reason the statis usd rate was casted as an object type columns rather than a float type. The good nes is that there is an easy fix for this

In [0]:
print(df[['goal','pledged','static_usd_rate']].info())

#changing column data type
df['static_usd_rate'] = df['static_usd_rate'].astype(float)

#converting columns to USD
for col in ['goal','pledged']:
   df[col+'_in_usd'] = df[col]*df['static_usd_rate']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315775 entries, 0 to 315774
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   goal             315775 non-null  float64
 1   pledged          315775 non-null  float64
 2   static_usd_rate  315775 non-null  object 
dtypes: float64(2), object(1)
memory usage: 7.2+ MB
None
time: 319 ms


## Removing \n from strings

In [0]:
df.columns

Index(['blurb', 'goal', 'pledged', 'state', 'disable_communication', 'country',
       'currency', 'deadline', 'state_changed_at', 'created_at', 'launched_at',
       'staff_pick', 'static_usd_rate', 'spotlight', 'creator.id',
       'creator.name', 'location.name', 'location.type', 'category.name',
       'category.slug', 'category.parent_name', 'profile.show_feature_image',
       'profile.should_show_feature_image_section', 'urls.web.project',
       'duration', 'goal_in_usd', 'pledged_in_usd'],
      dtype='object')

time: 3.57 ms


In [0]:
df.blurb = df.blurb.str.replace('\n','')

time: 356 ms


# Saving File

In [0]:
df['blurb'] = df['blurb'].str.replace('\r','-')
df['creator.name'] = df['blurb'].str.replace('\r','-')
df['blurb'] = df['blurb'].str.replace('\n','-')
df['creator.name'] = df['blurb'].str.replace('\n','-')
df['blurb'] = df['blurb'].str.replace('\t','-')
df['creator.name'] = df['blurb'].str.replace('\t','-')

time: 1.94 s


In [0]:
df.to_csv('/content/drive/My Drive/Data Science Projects/Kickstarter/clean_data.csv',sep ='\t')

time: 9.78 s


In [0]:
with open('/content/drive/My Drive/Data Science Projects/Kickstarter/clean_data.csv','r') as f:
  lista = []
  lines = f.readlines()
  for x,line in enumerate(lines):
    if len(line.split('\t')) != 28:
      lista.append((x-1,line))
lista

[(28158,
  '28158\t"På den svenska landsbygden finns hundratusentals obebodda hus. \tKuriöst nog är det svårt att hitta hus för familjer som vill flytta in."\t100000.0\t11750.0\tfailed\tFalse\tSE\tSEK\t2017-04-16 21:59:00\t2017-04-16 21:59:00\t2017-03-09 20:12:42\t2017-03-13 16:32:58\tFalse\t0.11097393\tFalse\t1094515878\t"På den svenska landsbygden finns hundratusentals obebodda hus. \tKuriöst nog är det svårt att hitta hus för familjer som vill flytta in."\tStockholm\tTown\tJournalism\tjournalism\tJournalism\tFalse\tTrue\thttps://www.kickstarter.com/projects/1094515878/obebodda-hus-pa-den-svenska-landsbygden?ref=discovery_category_newest\t34\t11097.393\t1303.9436775\n'),
 (55885,
  '55885\t"The\tnext\tproject on the\thorizon is\trenovation of the exterior\tfaçade of\tthe Spring Garden\tMill,\twhich is in need of paint and\trepair."\t2000.0\t641.0\tfailed\tFalse\tUS\tUSD\t2016-02-20 21:05:00\t2016-02-20 21:05:00\t2016-01-24 21:05:55\t2016-01-30 16:58:40\tFalse\t1.0\tFalse\t846214022\t

time: 1.19 s


In [0]:
pd.read_csv('/content/drive/My Drive/Data Science Projects/Kickstarter/clean_data.csv',sep = '\t')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,blurb,goal,pledged,state,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,staff_pick,static_usd_rate,spotlight,creator.id,creator.name,location.name,location.type,category.name,category.slug,category.parent_name,profile.show_feature_image,profile.should_show_feature_image_section,urls.web.project,duration,goal_in_usd,pledged_in_usd
0,0,The LoKi MUT is a small EDC multitool that inc...,2000.0,2474.0,successful,False,US,USD,2017-02-03 04:37:43,2017-02-03 04:37:43,2016-12-02 16:46:54,2017-01-04 04:37:43,False,1.000000,True,872303310,The LoKi MUT is a small EDC multitool that inc...,Dallas,Town,Gadgets,technology/gadgets,Technology,False,True,https://www.kickstarter.com/projects/872303310...,30,2000.000000,2474.000000
1,1,Handmade wooden ear plugs and stretchers in an...,500.0,0.0,failed,False,AT,EUR,2015-12-18 09:22:34,2015-12-18 09:22:35,2015-11-16 15:19:07,2015-11-18 09:22:34,False,1.066095,False,1539029550,Handmade wooden ear plugs and stretchers in an...,Micheldorf,Town,Woodworking,crafts/woodworking,Crafts,False,True,https://www.kickstarter.com/projects/153902955...,30,533.047335,0.000000
2,2,"Broke, But Not Broken LP #1",3000.0,4085.0,successful,False,US,USD,2020-01-13 17:01:20,2020-01-13 17:01:20,2019-12-09 20:28:22,2019-12-13 17:01:20,False,1.000000,True,1882152174,"Broke, But Not Broken LP #1",Worcester,Town,Music,music,Music,False,True,https://www.kickstarter.com/projects/themornin...,31,3000.000000,4085.000000
3,3,Maker Box is an AR play experience that combin...,10000.0,10353.0,successful,False,US,USD,2016-12-02 21:56:44,2016-12-02 21:56:44,2016-10-13 02:43:05,2016-11-02 20:56:44,False,1.000000,True,1031612808,Maker Box is an AR play experience that combin...,Los Gatos,Town,Children's Books,publishing/children's books,Publishing,False,True,https://www.kickstarter.com/projects/getquriou...,30,10000.000000,10353.000000
4,4,A children's book about a robots search for lo...,11000.0,14862.0,successful,False,US,USD,2016-06-06 04:01:00,2016-06-06 04:01:00,2016-04-17 03:40:01,2016-04-29 04:54:43,True,1.000000,True,1480798481,A children's book about a robots search for lo...,Oakland,Town,Children's Books,publishing/children's books,Publishing,False,True,https://www.kickstarter.com/projects/148079848...,37,11000.000000,14862.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315770,315770,An app for visualizing and realizing your drea...,14000.0,0.0,live,False,US,USD,2016-05-04 02:27:43,2016-03-20 02:27:43,2016-02-10 10:13:33,2016-03-20 02:27:43,False,1.000000,False,745045609,An app for visualizing and realizing your drea...,Baltimore,Town,Apps,technology/apps,Technology,False,True,https://www.kickstarter.com/projects/745045609...,45,14000.000000,0.000000
315771,315771,Recipes at your fingertips! simply scan or typ...,40000.0,13.0,live,False,US,USD,2016-04-30 19:26:43,2016-03-21 19:26:43,2016-03-18 19:16:21,2016-03-21 19:26:43,False,1.000000,False,123864669,Recipes at your fingertips! simply scan or typ...,Kalamazoo,Town,Apps,technology/apps,Technology,False,True,https://www.kickstarter.com/projects/123864669...,40,40000.000000,13.000000
315772,315772,Effortlessly take control of the temperature w...,60000.0,2055.5,live,False,DE,EUR,2016-05-07 17:38:22,2016-03-23 17:38:22,2016-03-06 14:01:37,2016-03-23 17:38:22,False,1.124342,False,313105663,Effortlessly take control of the temperature w...,Hamburg,Town,Wearables,technology/wearables,Technology,False,True,https://www.kickstarter.com/projects/313105663...,45,67460.527200,2311.085228
315773,315773,TurnUP is a Multi-Function Bluetooth Speaker. ...,77000.0,1954.0,live,False,US,USD,2016-06-10 16:45:47,2016-04-11 16:45:47,2016-02-23 06:16:23,2016-04-11 16:45:47,False,1.000000,False,1907931770,TurnUP is a Multi-Function Bluetooth Speaker. ...,Visalia,Town,Sound,technology/sound,Technology,False,True,https://www.kickstarter.com/projects/turnupspe...,60,77000.000000

time: 3.53 s


In [0]:

for col in df.columns:
  if df[col].dtype == 'object':
    result = df[col].apply(lambda x: True if '\t' in str(x) else False).sum()
    print(col,result)


blurb 0
state 0
country 0
currency 0
creator.name 0
location.name 0
location.type 0
category.name 0
category.slug 0
category.parent_name 0
profile.should_show_feature_image_section 0
urls.web.project 0
time: 2.02 s
